In [2]:
%load_ext autoreload
%autoreload 2

import torch
from src.architectures.feature_extractors.mobilenet import MobileNet
from src.module.multiclass import MulticlassImageClassifier
from src.architectures.head import MulticlassLinearClassificationHead
from pytorch_lightning import seed_everything

from torchinfo import summary

count_params = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)

seed_everything(42)
img = torch.rand(1, 3, 224, 224)

# summary(net, input_data=img)

Global seed set to 42


In [33]:
net = MobileNet(3, 1.0, "v3_small", True, False, True).cuda()

In [34]:
net.net(img.cuda()).shape

torch.Size([1, 1024])

In [35]:
summary(net, input_data=img, depth=3)

/home/shate/.cache/pypoetry/virtualenvs/image-classification-5tGgTKFW-py3.11/lib/python3.11/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/shate/.cache/pypoetry/virtualenvs/image-classification-5tGgTKFW-py3.11/lib/python3.11/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                                  Output Shape              Param #
ExternalFeatureExtractor                                [1, 1024]                 --
├─Sequential: 1-1                                       --                        --
│    └─Sequential: 2-1                                  [1, 576, 7, 7]            --
│    │    └─Conv2dNormActivation: 3-1                   [1, 16, 112, 112]         (464)
│    │    └─InvertedResidual: 3-2                       [1, 16, 56, 56]           (744)
│    │    └─InvertedResidual: 3-3                       [1, 24, 28, 28]           (3,864)
│    │    └─InvertedResidual: 3-4                       [1, 24, 28, 28]           (5,416)
│    │    └─InvertedResidual: 3-5                       [1, 40, 14, 14]           (13,736)
│    │    └─InvertedResidual: 3-6                       [1, 40, 14, 14]           (57,264)
│    │    └─InvertedResidual: 3-7                       [1, 40, 14, 14]           (57,264)
│    │    └─InvertedResidu